# 数据说明
预选赛训练数据和区域赛训练数据分别为不同10个风电场近一年的运行数据共30万余条，每15分钟采集一次，包括风速、风向、温度、湿度、气压和真实功率等，具体的数据字段中英文对应如下：

- WINDSPEED 预测风速
- WINDDIRECTION 风向
- TEMPERATURE 温度
- HUMIDITY 湿度
- PRESSURE 气压
- ROUND(A.WS,1) 实际风速
- ROUND(A.POWER,0) 实际功率

考虑到风电场的特殊性，不同风机间的地理位置也是序列预测的一个重要参考价值。不过本文更关注于与该项目类似的场景，多时间序列预测其实在各个企业里非常常见，比如生产企业不同电机的工况、不同产品在不同地区的销量预测，如果关联性不是特别大的情况，我们可以考虑用纯时间序列预测生成基线结果。


In [5]:
#导入需要的包
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import datetime
import paddlets
from paddlets import TSDataset
from paddlets import TimeSeries
from paddlets.models.forecasting import MLPRegressor, LSTNetRegressor
from paddlets.transform import Fill, StandardScaler
from paddlets.metrics import MSE, MAE
from paddlets.analysis import AnalysisReport, Summary
from paddlets.datasets.repository import get_dataset
import warnings
warnings.filterwarnings('ignore')

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

### 04-1.csv 和 04-2.csv是同一个风机多次导出的数据，需要合并
### 有.csv也有.xlsx，需要统一格式

In [ ]:
def data_preprocess(src_data_dir, dst_data_dir):
    files = os.listdir(src_data_dir)
    # 第一步，完成数据格式统一
    for f in files:
        # 获取文件路径
        data_file = os.path.join(src_data_dir, f)
        # 获取文件名后缀
        data_type = os.path.splitext(data_file)[-1]
        # 获取文件名前缀
        data_name = os.path.splitext(data_file)[0]
        # 如果是excel文件，进行转换
        if data_type == '.xlsx':
            # 需要特别注意的是，在读取excel文件时要指定空值的显示方式，否则会在保存时以字符“.”代替，影响后续的数据分析
            data_xls = pd.read_excel(data_file, index_col=0, na_values='')
            data_xls.to_csv(data_name + '.csv', encoding='utf-8')
            # 将xls文件存到目标目录
            os.sava(data_name + '.csv', dst_data_dir)
        else:
            # 将csv文件存到目标目录
            os.sava(data_file, dst_data_dir)

    # 第二步，完成多文件的合并，文件目录要重新更新一次
    files = os.listdir(dst_data_dir)
    for f in files:
        # 获取文件路径
        data_file = os.path.join(dst_data_dir, f)
        # 获取文件名前缀
        data_basename = os.path.basename(data_file)
        # 检查风机数据是否有多个数据文件
        if len(data_basename.split('-')) > 1:
            merge_list = []
            # 找出该风机的所有数据文件
            matches = [ f for f in files if (f.find(data_basename.split('-')[0] + '-') > -1)]
            for i in matches:
                # 读取风机这部分数据
                data_df = pd.read_csv(os.path.join(dst_data_dir, i), index_col=False, keep_default_na=False)
                merge_list.append(data_df)
            if len(merge_list) > 0:
                all_data = pd.concat(merge_list,axis=0,ignore_index=True).fillna(".")
                all_data.to_csv(os.path.join(dst_data_dir, data_basename.split('-')[0]+ '.csv'),index=False) 
            for i in matches:
                # 删除这部分数据文件
                os.remove(os.path.join(dst_data_dir, i))
            # 更新文件目录
            files = os.listdir(dst_data_dir)


In [ ]:
data_preprocess('../../data/raw/wind-power-forecast')

In [ ]:
os.listdir('../../data/processed/wind-power-forecast')

# 数据EDA

In [ ]:
# 读取一份数据文件，这里需要特别注意时间戳字段的格式指定
df = pd.read_csv('../../data/processed/wind-power-forecast/04.csv',parse_dates=['DATATIME'],infer_datetime_format=True,dayfirst=True)

# 查看04号风机的运行工况数据
df.head()

# 用paddlets完成初步数据分析

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.tail()

In [ ]:
# 因为数据批次不同，数据集中有一些时间戳重复的脏数据，送入paddlets前要进行处理，本赛题要求保留第一个数据
df.drop_duplicates(subset = ['DATATIME'],keep='first',inplace=True)

# 我们选取最后30天的风机工况数据进行可视化
target_cov_dataset = TSDataset.load_from_dataframe(
    df[-4*24*30:],
    time_col='DATATIME',
    target_cols='YD15',
    observed_cov_cols=['WINDSPEED', 'PREPOWER', 'WINDDIRECTION', 'TEMPERATURE',
       'HUMIDITY', 'PRESSURE', 'ROUND(A.WS,1)', 'ROUND(A.POWER,0)'],
    freq='15min',
    fill_missing_dates=False
)
# 由于不同指标的数值差异较大，在绘图时选取数值范围相近的组合分开绘制可视化图表
target_cov_dataset.plot(['WINDSPEED', 'TEMPERATURE', 'ROUND(A.WS,1)'])

In [ ]:
target_cov_dataset.plot(['WINDDIRECTION', 'HUMIDITY', 'PRESSURE'])

In [ ]:
target_cov_dataset.plot(['PREPOWER', 'ROUND(A.POWER,0)','YD15'])

In [ ]:
target_cov_dataset.summary()

In [ ]:
target_cov_dataset.max()

In [ ]:
report = AnalysisReport(target_cov_dataset, ["summary"])
# 加入时频分析后，需要较长的计算时间
# report = AnalysisReport(target_cov_dataset, ["summary","fft","stft","cwt"])
report.export_docx_report()

# 模型应用

In [ ]:
# 读取数据集

df = pd.read_csv('../../data/processed/wind-power-forecast/01.csv',parse_dates=['DATATIME'],infer_datetime_format=True,dayfirst=True,dtype={'WINDDIRECTION':np.float64, 'HUMIDITY':np.float64, 'PRESSURE':np.float64})

# 因为数据批次不同，数据集中有一些时间戳重复的脏数据，送入paddlets前要进行处理，本赛题要求保留第一个数据
df.drop_duplicates(subset = ['DATATIME'],keep='first',inplace=True)

df.info()

In [ ]:
# 数据集预处理，指定时间戳、目标值、观测值、频率，填充缺失值

target_cov_dataset = TSDataset.load_from_dataframe(
    df,
    time_col='DATATIME',
    target_cols=['ROUND(A.POWER,0)', 'YD15'],
    observed_cov_cols=['WINDSPEED', 'PREPOWER', 'WINDDIRECTION', 'TEMPERATURE',
       'HUMIDITY', 'PRESSURE', 'ROUND(A.WS,1)'],
    freq='15min',
    fill_missing_dates=True,
    fillna_method = 'pre'
)

In [ ]:
# 随机划分训练集、验证集、测试集，比例为70:9:21

train_dataset, val_test_dataset = target_cov_dataset.split(0.7)
val_dataset, test_dataset = val_test_dataset.split(0.3)
train_dataset.plot(add_data=[val_dataset,test_dataset], labels=['Val', 'Test'])

In [ ]:
# 按照时间划分训练集、验证集、测试集，比例为5-7月，8月，9月

_ , train_dataset = target_cov_dataset.split('2021-05-01 00:00:00')
train_dataset, val_test_dataset = train_dataset.split('2021-08-01 00:00:00')
val_dataset, test_dataset = val_test_dataset.split('2021-09-01 00:00:00')
# 最后一天的工况数据需要预测ROUND(A.POWER,0)和YD15两个字段
test_dataset, pred_dataset = test_dataset.split('2021-10-01 00:00:00')
train_dataset.plot(add_data=[val_dataset,test_dataset], labels=['Val', 'Test'])

In [ ]:
# 将以上数据集进行标准化处理

scaler = StandardScaler()
scaler.fit(train_dataset)
train_dataset_scaled = scaler.transform(train_dataset)
val_test_dataset_scaled = scaler.transform(val_test_dataset)
val_dataset_scaled = scaler.transform(val_dataset)
test_dataset_scaled = scaler.transform(test_dataset)

### 单模型预测

In [ ]:
lstm = LSTNetRegressor(
    in_chunk_len = 120 * 4,
    out_chunk_len = 24 * 4,
    max_epochs=10,
    optimizer_params= dict(learning_rate=1e-3),
)

In [ ]:
lstm.fit(train_dataset, val_dataset)

In [ ]:
subset_test_pred_dataset = lstm.predict(val_dataset)
subset_test_dataset, _ = test_dataset.split(len(subset_test_pred_dataset.target))

In [ ]:
# 模型评估
mae = MAE()
mae(subset_test_dataset, subset_test_pred_dataset)

In [ ]:
# 预测结果可视化
subset_test_dataset, _ = test_dataset.split(len(subset_test_pred_dataset.target))
subset_test_dataset.plot(add_data=subset_test_pred_dataset, labels=['Pred'])

In [ ]:
# 模型保存
lstm.save("lstm")

In [ ]:
# 也可以动转静保存
lstm.save("./model", network_model=True, dygraph_to_static=True)

In [ ]:
# 模型加载
from paddlets.models.model_loader import load
loaded_lstm = load("lstm")

In [ ]:
# 模型预测
result = loaded_lstm.predict(test_dataset)
result.to_dataframe()

In [ ]:
result.to_dataframe().to_csv('01.csv')

### 集成模型预测

In [ ]:
from paddlets.models.forecasting import MLPRegressor
from paddlets.models.forecasting import LSTNetRegressor
from paddlets.models.forecasting import RNNBlockRegressor

lstm_params = {
    'sampling_stride': 24*4,
    'eval_metrics':["mse", "mae"],
    'batch_size': 8,
    'max_epochs': 20,
    'patience': 10
}
rnn_params = {
    'sampling_stride': 24*4,
    'eval_metrics': ["mse", "mae"],
    'batch_size': 8,
    'max_epochs': 20,
    'patience': 10
}
mlp_params = {
    'sampling_stride': 24*4,
    'eval_metrics': ["mse", "mae"],
    'batch_size': 8,
    'max_epochs': 20,
    'patience': 10,
    'use_bn': True,
}

In [ ]:
from paddlets.ensemble import StackingEnsembleForecaster

reg = StackingEnsembleForecaster(
in_chunk_len= 120 * 4,
out_chunk_len= 24 * 4,
skip_chunk_len=0,
estimators=[(LSTNetRegressor, lstm_params),(RNNBlockRegressor, rnn_params), (MLPRegressor, mlp_params)])


In [ ]:
# 模型拟合
reg.fit(train_dataset, val_dataset)

In [ ]:
subset_test_pred_dataset = reg.predict(val_dataset)
subset_test_dataset, _ = test_dataset.split(len(subset_test_pred_dataset.target))

In [ ]:
# 模型评估
mae = MAE()
mae(subset_test_dataset, subset_test_pred_dataset)

In [ ]:
subset_test_dataset, _ = test_dataset.split(len(subset_test_pred_dataset.target))
subset_test_dataset.plot(add_data=subset_test_pred_dataset, labels=['Pred'])

In [ ]:
reg.save("reg")

In [ ]:
# 集成模型加载
loaded_model0 = load("./reg/paddlets-ensemble-model0")
loaded_model1 = load("./reg/paddlets-ensemble-model1")
loaded_model2 = load("./reg/paddlets-ensemble-model2")

In [ ]:
# 输出集成模型预测结果
loaded_model0.predict(test_dataset).to_dataframe()*0.2 + loaded_model1.predict(test_dataset).to_dataframe()*0.4 + loaded_model2.predict(test_dataset).to_dataframe()*0.4

In [ ]:
# 保存集成模型预测结果
# (loaded_model0.predict(test_dataset).to_dataframe()*0.2 + loaded_model1.predict(test_dataset).to_dataframe()*0.4 + loaded_model2.predict(test_dataset).to_dataframe()*0.4).to_csv('01.csv')

### 回测

In [ ]:
from paddlets.utils import backtest
from paddlets.metrics import MAE
mae, ts_pred = backtest(data=test_dataset,
                model=lstm,
                start=0.5,  # the point after "start" as the first point
                metric=MAE(),
                return_predicts=True
                )

In [ ]:
mae

4.5 模型保存与提交说明

从赛题给的风机数据中我们可以发现，不同风机数据起始时间、预测日期均不一致。因此，选手需要逐个风机进行预测并生成提交文件。从另一个角度说，不同风机可以使用同一个时序模型、超参数，也可以使用不同的模型、超参数，选手可以自行设置。